In [1]:
libraries = c("dplyr", "tidyverse", "data.table", "readxl")
for(x in libraries) {library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE)}

theme_set(theme_bw())

── Attaching packages ─────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0     ✔ purrr   0.3.5
✔ tibble  3.1.8     ✔ stringr 1.4.1
✔ tidyr   1.2.1     ✔ forcats 0.5.2
✔ readr   2.1.3     
── Conflicts ────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
path <- "../data/flight/all_region/"

options(warn=-1)
list.files(path = path, pattern = "*xlsx") -> file_list
read_xlsx_files <- function(x){
    df <- suppressMessages(read_xlsx(path = paste0(path, x), skip = 5))
    df[-1, c(1,2,27)] -> df_2019
    colnames(df_2019) <- c("destination", "series", "v2019")
    return(df_2019)}
lapply(file_list, read_xlsx_files) -> df_list

for(i in 1:length(df_list)){
    colnames(df_list[[i]]) <- c("destination", "series", substr(file_list[i],1,nchar(file_list[i])-5))
}

options(warn=0)

In [3]:
merge.all <- function(x, y) {merge(x, y, all=TRUE, by=c("destination", "series"))}
output <- Reduce(merge.all, df_list)

output %>% filter(!is.na(destination)) -> output

In [4]:
output %>% filter(destination %in% c("Czechia", "Czech Republic")) -> temp
unique(temp$series) -> series_list
Czech_list <- list()
for(i in 1:length(series_list)){
    temp %>% filter(series==series_list[i]) -> temp; temp[is.na(temp)] <- 0
    temp %>% dplyr::select(-c("destination", "series")) -> temp_col
    as.data.frame(colSums(temp_col))%>% t() %>% as.data.frame() %>%
    mutate(destination=c("Czech Republic"), series=series_list[i]) -> temp_all; rownames(temp_all) <- NULL
    temp_all -> Czech_list[[i]]
}

do.call(rbind, Czech_list) -> Czech_output

In [5]:
output %>% filter(destination %in% c("Türkiye", "Turkey")) -> temp
unique(temp$series) -> series_list
Turkey_list <- list()
for(i in 1:length(series_list)){
    temp %>% filter(series==series_list[i]) -> temp; temp[is.na(temp)] <- 0
    temp %>% dplyr::select(-c("destination", "series")) -> temp_col
    as.data.frame(colSums(temp_col))%>% t() %>% as.data.frame() %>%
    mutate(destination=c("Turkey"), series=series_list[i]) -> temp_all; rownames(temp_all) <- NULL
    temp_all -> Turkey_list[[i]]
}

do.call(rbind, Turkey_list) -> Turkey_output

In [6]:
output %>% filter(!(destination %in% c("END", "Türkiye", "Turkey", "Czechia", "Czech Republic"))) %>%
mutate(destination=case_when(destination==c("Curaçao")~c("Curacao"), 
                             destination==c("Côte d\'Ivoire")~c("Cote d'Ivoire"), 
                             destination==c("Taiwan Province of China")~c("Taiwan, Province of China"), 
                             destination==c("Saint Vincent and the Grenadines")~c("Saint Vincent and The Grenadines"),
                             destination==c("Palestine")~c("Palestine, State of"),
                             TRUE~destination)) -> output

rbind(output, Turkey_output, Czech_output) -> output

In [7]:
## coverting NAs to zero in the flight data
output[is.na(output)] <- 0
output %>% mutate(total_vol = rowSums(output[,3:ncol(output)])) %>% 
filter(total_vol > 0) %>% dplyr::select(-total_vol) -> output

In [8]:
## selecting the biggest value throughout the series
colMax <- function(data) sapply(data, max, na.rm = TRUE)

new_list <- list()
unique(output$destination) -> country_list


for (g in 1:length(country_list)){
    output %>% filter(destination==country_list[g]) -> temp
    as.data.frame(t(colMax(temp)))-> new_list[[g]]
}

            
do.call("rbind", new_list) -> flight_matrix
write.csv(flight_matrix, "../data/flight/flight_matrix.csv")     

In [16]:
flight_matrix %>% filter(destination==("Palestine, State of")) -> temp
temp[, 3:215] <- sapply(temp[, 3:215], as.numeric)
temp %>% dplyr::select_if(~ !is.numeric(.) || sum(.) != 0)

destination,series,Israel
<chr>,<chr>,<dbl>
"Palestine, State of",THSN,85090


In [10]:
# ## selecting the series of data with prioritization
# new_list <- list()
# unique(output$destination) -> country_list

# for (g in 1:length(country_list)){
#     output %>% filter(destination==country_list[g]) -> temp
#     if(nrow(temp)>=2){
#         if(any(temp$series==c("VFR"))){temp %>% filter(series==c("VFR")) -> new_list[[g]]}
#         else if(any(temp$series==c("VFN")) && !(temp$series %in% c("VFR")))
#         {temp %>% filter(series==c("VFN")) -> new_list[[g]]}
#         else if(any(temp$series==c("TFR")) && !(temp$series %in% c("VFR", "VFN")))
#         {temp %>% filter(series==c("TFR")) -> new_list[[g]]}
#         else if(any(temp$series==c("TFN")) && !(temp$series %in% c("VFR", "VFR", "TFR")))
#         {temp %>% filter(series==c("TFN")) -> new_list[[g]]}
#         else if(any(temp$series==c("TCER")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN")))
#         {temp %>% filter(series==c("TCER")) -> new_list[[g]]}
#         else if(any(temp$series==c("TCEN")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER")))
#         {temp %>% filter(series==c("TCEN")) -> new_list[[g]]}
#         else if(any(temp$series==c("THSR")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER", "TCEN")))
#         {temp %>% filter(series==c("THSR")) -> new_list[[g]]}
#         else if(any(temp$series==c("THSN")) && !(temp$series %in% c("VFR", "VFR", "TFR", "TFN", "TCER", "TCEN", "THSR")))
#         {temp %>% filter(series==c("THSN")) -> new_list[[g]]}
#     }
#     else{temp -> new_list[[g]]}
# }
            
# do.call("rbind", new_list) -> flight_matrix
# # flight_matrix %>% 
# write.csv(flight_matrix, "../data/flight/flight_matrix.csv")     

In [11]:
setdiff(flight_matrix$destination, colnames(flight_matrix[,3:ncol(flight_matrix)]))
setdiff(colnames(flight_matrix[,3:ncol(flight_matrix)]), flight_matrix$destination)

[1] "Cabo Verde"                   "Eswatini"                    
[3] "Reunion"                      "Sint Maarten (Dutch part)"   
[5] "Taiwan, Province of China"    "Tanzania, United Republic of"

[1] "Afghanistan"                          
 [2] "American Samoa"                       
 [3] "Botswana"                             
 [4] "British Virgin Islands"               
 [5] "Burundi"                              
 [6] "Cape Verde"                           
 [7] "Chad"                                 
 [8] "Congo, Democratic Republic of the"    
 [9] "Congo"                                
[10] "Djibouti"                             
[11] "Equatorial Guinea"                    
[12] "Eritrea"                              
[13] "Gabon"                                
[14] "Ghana"                                
[15] "Guadeloupe"                           
[16] "Guinea-Bissau"                        
[17] "Guinea"                               
[18] "Iraq"                                 
[19] "Korea, Democratic Peoples Republic of"
[20] "Liberia"                              
[21] "Libyan Arab Jamahiriya"               
[22] "Malawi"                               
[23] "Marshall Islands"                     
[24] "Mauritania"                           
[25] "Micronesia (Federated States Of)"     
[26] "Nauru"                                
[27] "Nigeria"                              
[28] "Niue"                                 
[29] "Pakistan"                             
[30] "Sao Tome and Principe"                
[31] "Senegal"                              
[32] "Somalia"                              
[33] "Sudan"                                
[34] "Suriname"                             
[35] "Swaziland"                            
[36] "Turkmenistan"                         
[37] "Uganda"                               
[38] "Venezuela, Bolivarian Republic of"    
[39] "Yemen"